In [ ]:
from google.colab import drive
import pickle
from tqdm import tqdm
import gensim
import numpy as np
import time
from sklearn import linear_model
from sklearn import svm
from sklearn import ensemble
import xgboost
from keras.layers import *
from keras import layers, models, optimizers
from keras.preprocessing import text, sequence
from sklearn import preprocessing
from sklearn import naive_bayes
from sklearn import metrics
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Load data
drive.mount('/content/drive', force_remount=True)
DATA_PATH = "/content/drive/My Drive/NLP"

x_data = pickle.load(open(DATA_PATH + '/Processed Data/x_data.pkl', 'rb'))
y_data = pickle.load(open(DATA_PATH + '/Processed Data/y_data.pkl', 'rb'))

x_test = pickle.load(open(DATA_PATH + '/Processed Data/x_test.pkl', 'rb'))
y_test = pickle.load(open(DATA_PATH + '/Processed Data/y_test.pkl', 'rb'))

x_data_vectors = pickle.load(open(DATA_PATH + '/Processed Data/x_data_vectors.pkl', 'rb'))
x_test_vectors = pickle.load(open(DATA_PATH + '/Processed Data/x_test_vectors.pkl', 'rb'))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
def get_corpus(documents):
    corpus = []
    
    for i in tqdm(range(len(documents))):
        doc = documents[i]
        
        words = doc.split(' ')
        tagged_document = gensim.models.doc2vec.TaggedDocument(words, [i])
        
        corpus.append(tagged_document)
        
    return corpus

In [ ]:
train_corpus = get_corpus(x_data)

100%|██████████| 46668/46668 [00:09<00:00, 5015.23it/s]


In [ ]:
test_corpus = get_corpus(x_test)

100%|██████████| 49284/49284 [00:03<00:00, 13011.95it/s]


In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=2, epochs=40)
model.build_vocab(train_corpus)

In [ ]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 1h 1min 3s, sys: 35.7 s, total: 1h 1min 39s
Wall time: 32min 23s


In [ ]:
x_data_vectors = []
for x in train_corpus:
    vector = model.infer_vector(x.words)
    x_data_vectors.append(vector)

In [ ]:
x_test_vectors = []
for x in test_corpus:
    vector = model.infer_vector(x.words)
    x_test_vectors.append(vector)

In [ ]:
classifier = create_dnn_model()
train_model(classifier=classifier, x_data=np.array(x_data_vectors), y_data=y_data_n, x_test=(x_test_vectors), y_test=y_test_n, is_neuralnet=True, n_epochs=100)

In [ ]:
# Train Model
def train_model(classifier, x_data, y_data, x_test, y_test, is_neuralnet=False, n_epochs=100):
    x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=42)

    if is_neuralnet:
        classifier.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=n_epochs, batch_size=512)

        val_predictions = classifier.predict(x_val)
        test_predictions = classifier.predict(x_test)
        val_predictions = val_predictions.argmax(axis=-1)
        test_predictions = test_predictions.argmax(axis=-1)
    else:
        classifier.fit(x_train, y_train)

        train_predictions = classifier.predict(x_train)
        val_predictions = classifier.predict(x_val)
        test_predictions = classifier.predict(x_test)

    # Evaluation

    print("> Validation Accuracy: ", metrics.accuracy_score(val_predictions, y_val))

    print("> Test Accuracy: ", metrics.accuracy_score(test_predictions, y_test))

    print("> Validation Precision: ", metrics.precision_score(y_val, val_predictions, average='macro'))

    print("> Test Precision: ", metrics.precision_score(y_test, test_predictions, average='macro'))

    print("> Validation Recall: ", metrics.recall_score(y_val, val_predictions, average='macro'))

    print("> Test Recall: ", metrics.recall_score(y_test, test_predictions, average='macro'))

    print("> Validation F1 Score: ", metrics.f1_score(y_val, val_predictions, average='macro'))

    print("> Test F1 Score: ", metrics.f1_score(y_test, test_predictions, average='macro'))

In [ ]:
scaler = MinMaxScaler()

In [ ]:
x_data_vectors = sklearn.preprocessing.MinMaxScaler().fit_transform(x_data_vectors)

In [ ]:
x_test_vectors = sklearn.preprocessing.MinMaxScaler().fit_transform(x_test_vectors)

In [ ]:
x_data_vectors = pickle.load(open(DATA_PATH + '/Processed Data/x_data_vectors.pkl', 'rb'))
x_test_vectors = pickle.load(open(DATA_PATH + '/Processed Data/x_test_vectors.pkl', 'rb'))

In [ ]:
# Naive Bayes
# Naive Bayes (Multinomial)
print("\n\n----- Naive Bayes (Multinomial) -----")
nbm_start = time.time()
train_model(naive_bayes.MultinomialNB(), x_data_vectors, y_data, x_test_vectors, y_test, is_neuralnet=False)
nbm_stop = time.time()
print(f"> Training time: {nbm_stop - nbm_start}s")



----- Naive Bayes (Multinomial) -----
> Validation Accuracy:  0.39725733876151703
> Test Accuracy:  0.3399277656034413
> Validation Precision:  0.697829394851257


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


> Test Precision:  0.576400438103393
> Validation Recall:  0.30380621121727547
> Test Recall:  0.31529412175308646
> Validation F1 Score:  0.26152633109220047
> Test F1 Score:  0.2099577998809161
> Training time: 1.0663821697235107s


In [ ]:
# Naive Bayes (Bernoulli)
print("\n\n----- Naive Bayes (Bernoulli) -----")
nbb_start = time.time()
train_model(naive_bayes.BernoulliNB(), x_data_vectors, y_data, x_test_vectors, y_test, is_neuralnet=False)
nbb_stop = time.time()
print(f"> Training time: {nbb_stop - nbb_start}s")



----- Naive Bayes (Bernoulli) -----
> Validation Accuracy:  0.7422326976644525
> Test Accuracy:  0.7440142845548251
> Validation Precision:  0.7409103085653176
> Test Precision:  0.6591482069879818
> Validation Recall:  0.7415870204058723
> Test Recall:  0.7273468127943074
> Validation F1 Score:  0.7385712629061151
> Test F1 Score:  0.6775109456765621
> Training time: 1.9441683292388916s


In [ ]:
# Logistic Regression
print("\n\n----- Logistic Regression -----")
lr_start = time.time()
train_model(linear_model.LogisticRegression(), x_data_vectors, y_data, x_test_vectors, y_test, is_neuralnet=False)
lr_stop = time.time()
print(f"> Training time: {lr_stop - lr_start}s")



----- Logistic Regression -----


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


> Validation Accuracy:  0.8566530962074138
> Test Accuracy:  0.8206720233747261
> Validation Precision:  0.8533391494363355
> Test Precision:  0.7366886083282338
> Validation Recall:  0.8514459904253407
> Test Recall:  0.8192151348071233
> Validation F1 Score:  0.8520104095856215
> Test F1 Score:  0.7594212850004193
> Training time: 15.106202602386475s


In [ ]:
# Support Vector Machine
print("\n\n----- Support Vector Machine -----")
svm_start = time.time()
train_model(svm.SVC(), x_data_vectors, y_data, x_test_vectors, y_test, is_neuralnet=False)
svm_stop = time.time()
print(f"> Training time: {svm_stop - svm_start}s")



----- Support Vector Machine -----
> Validation Accuracy:  0.889222198414399
> Test Accuracy:  0.8675635094554014
> Validation Precision:  0.8897065163011809
> Test Precision:  0.8054405875517278
> Validation Recall:  0.8843261680997976
> Test Recall:  0.8568300992186862
> Validation F1 Score:  0.8863011368842992
> Test F1 Score:  0.8230029419196178
> Training time: 1850.1757400035858s


In [ ]:
# Random Forest Classifier
print("\n\n----- Random Forest Classifier -----")
rfc_start = time.time()
train_model(ensemble.RandomForestClassifier(), x_data_vectors, y_data, x_test_vectors, y_test, is_neuralnet=False)
rfc_stop = time.time()
print(f"> Training time: {rfc_stop - rfc_start}s")



----- Random Forest Classifier -----
> Validation Accuracy:  0.6638097278765802
> Test Accuracy:  0.639923707491275
> Validation Precision:  0.7162736799116759
> Test Precision:  0.607439964722171
> Validation Recall:  0.6201510019515868
> Test Recall:  0.59580885606317
> Validation F1 Score:  0.6334475093701379
> Test F1 Score:  0.5531664114009653
> Training time: 105.60199069976807s


In [ ]:
# XGBoost
print("\n\n----- XGBoost -----")
xgb_start = time.time()
train_model(xgboost.XGBClassifier(), np.array(x_data_vectors), y_data, np.array(x_test_vectors), y_test, is_neuralnet=False)
xgb_stop = time.time()
print(f"> Training time: {xgb_stop - xgb_start}s")



----- XGBoost -----
> Validation Accuracy:  0.7278765802442683
> Test Accuracy:  0.6945458972485999
> Validation Precision:  0.7324626827905019
> Test Precision:  0.6125162229625599
> Validation Recall:  0.7113509452532584
> Test Recall:  0.6807360158202963
> Validation F1 Score:  0.7191475216348818
> Test F1 Score:  0.6218518648363062
> Training time: 1115.231874704361s


In [ ]:
# Deep Neural Network
dnn_start = time.time()
encoder = preprocessing.LabelEncoder()
y_data_n = encoder.fit_transform(y_data)
y_test_n = encoder.fit_transform(y_test)

def create_dnn_model():
    input_layer = Input(shape=(300,))
    layer = Dense(1024, activation='relu')(input_layer)
    layer = Dense(1024, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    output_layer = Dense(13, activation='softmax')(layer)

    classifier = models.Model(input_layer, output_layer)
    classifier.compile(optimizer=optimizers.Adam(
    ), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return classifier

classifier = create_dnn_model()
print("\n\n----- Deep Neural Network -----")
train_model(classifier=classifier, x_data=np.array(x_data_vectors), y_data=y_data_n, x_test=np.array(x_test_vectors), y_test=y_test_n, is_neuralnet=True, n_epochs=100)
dnn_stop = time.time()
print(f"> Training time: {dnn_stop - dnn_start}s")



----- Deep Neural Network -----
Train on 42001 samples, validate on 4667 samples
Epoch 1/100
42001/42001 [==============================] - 11s 258us/step - loss: 0.6623 - accuracy: 0.7916 - val_loss: 0.4699 - val_accuracy: 0.8492
Epoch 2/100
42001/42001 [==============================] - 10s 247us/step - loss: 0.3220 - accuracy: 0.8939 - val_loss: 0.4630 - val_accuracy: 0.8498
Epoch 3/100
42001/42001 [==============================] - 10s 248us/step - loss: 0.2204 - accuracy: 0.9261 - val_loss: 0.4205 - val_accuracy: 0.8708
Epoch 4/100
42001/42001 [==============================] - 11s 250us/step - loss: 0.1315 - accuracy: 0.9570 - val_loss: 0.4812 - val_accuracy: 0.8642
Epoch 5/100
42001/42001 [==============================] - 11s 257us/step - loss: 0.0945 - accuracy: 0.9692 - val_loss: 0.5493 - val_accuracy: 0.8584
Epoch 6/100
42001/42001 [==============================] - 10s 248us/step - loss: 0.0809 - accuracy: 0.9741 - val_loss: 0.5965 - val_accuracy: 0.8612
Epoch 7/100
42001

In [ ]:
# Recurrent Neural Network - LSTM
def create_lstm_model():
    input_layer = Input(shape=(300,))

    layer = Reshape((10, 30))(input_layer)
    layer = LSTM(128, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)

    output_layer = Dense(13, activation='softmax')(layer)

    classifier = models.Model(input_layer, output_layer)

    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return classifier

lstm_start = time.time()
classifier = create_lstm_model()
print("\n\n----- Recurrent Neural Network - LSTM -----")
train_model(classifier=classifier, x_data=np.array(x_data_vectors), y_data=y_data_n, x_test=np.array(x_test_vectors), y_test=y_test_n, is_neuralnet=True, n_epochs=100)
lstm_stop = time.time()
print(f"> Training time: {lstm_stop - lstm_start}s")



----- Recurrent Neural Network - LSTM -----
Train on 42001 samples, validate on 4667 samples
Epoch 1/100
42001/42001 [==============================] - 12s 292us/step - loss: 1.7277 - accuracy: 0.4394 - val_loss: 1.3338 - val_accuracy: 0.5762
Epoch 2/100
42001/42001 [==============================] - 12s 280us/step - loss: 1.1079 - accuracy: 0.6409 - val_loss: 1.0036 - val_accuracy: 0.6758
Epoch 3/100
42001/42001 [==============================] - 12s 281us/step - loss: 0.9027 - accuracy: 0.7042 - val_loss: 0.9839 - val_accuracy: 0.6833
Epoch 4/100
42001/42001 [==============================] - 12s 277us/step - loss: 0.7722 - accuracy: 0.7456 - val_loss: 0.8852 - val_accuracy: 0.7114
Epoch 5/100
42001/42001 [==============================] - 12s 279us/step - loss: 0.7016 - accuracy: 0.7650 - val_loss: 0.7047 - val_accuracy: 0.7628
Epoch 6/100
42001/42001 [==============================] - 12s 275us/step - loss: 0.6351 - accuracy: 0.7863 - val_loss: 0.8051 - val_accuracy: 0.7424
Epoch

In [ ]:
# Recurrent Neural Network - GRU
def create_gru_model():
    input_layer = Input(shape=(300,))

    layer = Reshape((10, 30))(input_layer)
    layer = GRU(128, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)

    output_layer = Dense(13, activation='softmax')(layer)

    classifier = models.Model(input_layer, output_layer)

    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return classifier

gru_start = time.time()
classifier = create_gru_model()
print("\n\n----- Recurrent Neural Network - GRU -----")
train_model(classifier=classifier, x_data=np.array(x_data_vectors), y_data=y_data_n, x_test=np.array(x_test_vectors), y_test=y_test_n, is_neuralnet=True, n_epochs=100)
gru_stop = time.time()
print(f"> Training time: {gru_stop - gru_start}s")



----- Recurrent Neural Network - GRU -----
Train on 42001 samples, validate on 4667 samples
Epoch 1/100
42001/42001 [==============================] - 12s 281us/step - loss: 1.7217 - accuracy: 0.4451 - val_loss: 1.4400 - val_accuracy: 0.5316
Epoch 2/100
42001/42001 [==============================] - 11s 270us/step - loss: 1.1264 - accuracy: 0.6347 - val_loss: 0.9852 - val_accuracy: 0.6852
Epoch 3/100
42001/42001 [==============================] - 11s 267us/step - loss: 0.8736 - accuracy: 0.7140 - val_loss: 0.8469 - val_accuracy: 0.7292
Epoch 4/100
42001/42001 [==============================] - 11s 264us/step - loss: 0.7568 - accuracy: 0.7504 - val_loss: 0.7959 - val_accuracy: 0.7390
Epoch 5/100
42001/42001 [==============================] - 11s 261us/step - loss: 0.6737 - accuracy: 0.7757 - val_loss: 0.7309 - val_accuracy: 0.7628
Epoch 6/100
42001/42001 [==============================] - 11s 266us/step - loss: 0.6082 - accuracy: 0.7942 - val_loss: 0.6745 - val_accuracy: 0.7795
Epoch 

In [ ]:
# Bidirectional RNN
def create_brnn_model():
    input_layer = Input(shape=(300,))

    layer = Reshape((10, 30))(input_layer)
    layer = Bidirectional(GRU(128, activation='relu'))(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)

    output_layer = Dense(13, activation='softmax')(layer)

    classifier = models.Model(input_layer, output_layer)

    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return classifier

brnn_start = time.time()
classifier = create_brnn_model()
print("\n\n----- Bidirectional RNN -----")
train_model(classifier=classifier, x_data=np.array(x_data_vectors), y_data=y_data_n, x_test=np.array(x_test_vectors), y_test=y_test_n, is_neuralnet=True, n_epochs=100)
brnn_stop = time.time()
print(f"> Training time: {brnn_stop - brnn_start}s")



----- Bidirectional RNN -----
Train on 42001 samples, validate on 4667 samples
Epoch 1/100
42001/42001 [==============================] - 21s 495us/step - loss: 1.4412 - accuracy: 0.5337 - val_loss: 0.9187 - val_accuracy: 0.7075
Epoch 2/100
42001/42001 [==============================] - 20s 472us/step - loss: 0.7899 - accuracy: 0.7460 - val_loss: 0.8700 - val_accuracy: 0.7154
Epoch 3/100
42001/42001 [==============================] - 20s 472us/step - loss: 0.6327 - accuracy: 0.7935 - val_loss: 0.7696 - val_accuracy: 0.7532
Epoch 4/100
42001/42001 [==============================] - 20s 472us/step - loss: 0.5446 - accuracy: 0.8213 - val_loss: 0.6993 - val_accuracy: 0.7814
Epoch 5/100
42001/42001 [==============================] - 20s 470us/step - loss: 0.4775 - accuracy: 0.8413 - val_loss: 0.7063 - val_accuracy: 0.7703
Epoch 6/100
42001/42001 [==============================] - 20s 476us/step - loss: 0.4395 - accuracy: 0.8541 - val_loss: 0.7258 - val_accuracy: 0.7746
Epoch 7/100
42001/4

In [ ]:
# Recurrent Convolutional Neural Network
def create_rcnn_model():
    input_layer = Input(shape=(300,))

    layer = Reshape((10, 30))(input_layer)
    layer = Bidirectional(
        GRU(128, activation='relu', return_sequences=True))(layer)
    layer = Convolution1D(100, 3, activation="relu")(layer)
    layer = Flatten()(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)

    output_layer = Dense(13, activation='softmax')(layer)

    classifier = models.Model(input_layer, output_layer)
    classifier.summary()
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return classifier

rcnn_start = time.time()
classifier = create_rcnn_model()
print("\n\n----- Recurrent Convolutional Neural Network -----")
train_model(classifier=classifier, x_data=np.array(x_data_vectors), y_data=y_data_n, x_test=np.array(x_test_vectors), y_test=y_test_n, is_neuralnet=True, n_epochs=100)
rcnn_stop = time.time()
print(f"> Training time: {rcnn_stop - rcnn_start}s")

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 300)               0         
_________________________________________________________________
reshape_4 (Reshape)          (None, 10, 30)            0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 10, 256)           122112    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 8, 100)            76900     
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 512)               410112    
_________________________________________________________________
dense_18 (Dense)             (None, 512)               2626